# IMPORTS

In [37]:
import requests
import pandas as pd

from datetime import datetime

from bs4 import BeautifulSoup

## Helper functions

# DATA EXTRATION

In [4]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:106.0) Gecko/20100101 Firefox/106.0'}

page = requests.get(url, headers=headers)

In [7]:
soup = BeautifulSoup(page.text, 'html.parser')

In [9]:
products = soup.find('ul', class_='products-listing small')

In [18]:
product_list = products.find_all('article', class_='hm-product-item')

# product id
product_id = [p.get('data-articlecode') for p in product_list]

# product category
product_category = [p.get('data-category') for p in product_list]

In [30]:
# product name
product_list = products.find_all('a', class_='link')
product_name = [p.get_text() for p in product_list]

In [28]:
# price
product_list = products.find_all('span', class_='price regular')
product_price = [p.get_text() for p in product_list]

In [ ]:
# product color

In [ ]:
# product composition

In [40]:
data = pd.DataFrame([product_id, product_category, product_name, product_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [41]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,1024256007,men_jeans_slim,Slim Jeans,$ 19.99,2022-10-27 11:37:22
1,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-10-27 11:37:22
2,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-10-27 11:37:22
3,1071707008,men_jeans_relaxed,Relaxed Jeans,$ 29.99,2022-10-27 11:37:22
4,1024711006,men_jeans_slim,Slim Jeans,$ 29.99,2022-10-27 11:37:22
